# Prudential Life Insurance Assessment

In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('seaborn-whitegrid')

from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Get the Data

In [2]:
df1 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train1.csv')
df2 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train2.csv')
df3 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train3.csv')
df4 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train4.csv')
df5 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train5.csv')
df6 = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_train6.csv')
dft = pd.read_csv('E:/AdvAnal/python_103/1_Data/df_test.csv')

In [3]:
train = pd.concat([df1,df2,df3,df5,df6],axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49381 entries, 0 to 9380
Columns: 127 entries, y to WT_
dtypes: int64(127)
memory usage: 48.2 MB


In [4]:
test = df4
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 127 entries, y to WT_
dtypes: int64(127)
memory usage: 9.7 MB


In [5]:
df = pd.concat([df1,df2,df3,df4,df5,df6],axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59381 entries, 0 to 9380
Columns: 127 entries, y to WT_
dtypes: int64(127)
memory usage: 58.0 MB


In [6]:
sub = dft
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 127 entries, Id to WT_
dtypes: int64(127)
memory usage: 19.2 MB


# Clean the Data

In [7]:
cat_feats = ['BMI_', 'EI_1_', 'EI_2_', 'EI_3_', 'EI_4_', 'EI_5_', 'EI_6_', 'FH_1_', 'FH_2_', 'FH_3_', 'FH_4_', 'FH_5_', 
             'HT_', 'IAGE_', 'IH_1_', 'IH_2_', 'IH_3_', 'IH_4_', 'IH_5_', 'IH_7_', 'IH_8_', 'IH_9_', 
             'II_1_', 'II_2_', 'II_3_', 'II_4_', 'II_5_', 'II_6_', 'II_7_', 'MH_1_', 'MH_2_', 'MH_3_', 'MH_4_', 'MH_5_', 'MH_6_', 
             'MH_7_', 'MH_8_', 'MH_9_', 'MH_10_', 'MH_11_', 'MH_12_', 'MH_13_', 'MH_14_', 'MH_15_', 'MH_16_', 'MH_17_', 'MH_18_', 
             'MH_19_', 'MH_20_', 'MH_21_', 'MH_22_', 'MH_23_', 'MH_24_', 'MH_25_', 'MH_26_', 'MH_27_', 'MH_28_', 'MH_29_', 
             'MH_30_', 'MH_31_', 'MH_32_', 'MH_33_', 'MH_34_', 'MH_35_', 'MH_36_', 'MH_37_', 'MH_38_', 'MH_39_', 'MH_40_', 'MH_41_', 
             'MK_1_', 'MK_2_', 'MK_3_', 'MK_4_', 'MK_5_', 'MK_6_', 'MK_7_', 'MK_8_', 'MK_9_', 'MK_10_', 'MK_11_', 'MK_12_', 
             'MK_13_', 'MK_14_', 'MK_15_', 'MK_16_', 'MK_17_', 'MK_18_', 'MK_19_', 'MK_20_', 'MK_21_', 'MK_22_', 'MK_23_', 'MK_24_', 
             'MK_25_', 'MK_26_', 'MK_27_', 'MK_28_', 'MK_29_', 'MK_30_', 'MK_31_', 'MK_32_', 'MK_33_', 'MK_34_', 'MK_35_', 'MK_36_', 
             'MK_37_', 'MK_38_', 'MK_39_', 'MK_40_', 'MK_41_', 'MK_42_', 'MK_43_', 'MK_44_', 'MK_45_', 'MK_46_', 'MK_47_', 'MK_48_', 
             'PI_1_', 'PI_2_', 'PI_3_', 'PI_4_', 'PI_5_', 'PI_6_', 'PI_7_', 'WT_']

In [8]:
y_train = train['y'].copy()
x_train = train.drop('y', 1).copy()
x_train = pd.get_dummies(x_train,columns=cat_feats,drop_first=True).copy()

In [9]:
y_test = test['y'].copy()
x_test = test.drop('y', 1).copy()
x_test = pd.get_dummies(x_test,columns=cat_feats,drop_first=True).copy()

In [10]:
y_df = df['y'].copy()
x_df = df.drop('y', 1).copy()
x_df = pd.get_dummies(x_df,columns=cat_feats,drop_first=True).copy()

In [11]:
y_sub = sub['Id'].copy()
x_sub = sub.drop('Id', 1).copy()
x_sub = pd.get_dummies(x_sub,columns=cat_feats,drop_first=True).copy()

# XG Boost

In [12]:
xgb = XGBClassifier(objective='binary:logistic', n_estimators=160, max_depth=12, seed=101)
xgb.fit(x_train, y_train)
y_pred=xgb.predict(x_test)

In [13]:
test_calc = pd.concat([pd.DataFrame(y_test).reset_index(drop=True),pd.DataFrame(y_pred).reset_index(drop=True)],axis=1)
test_calc.rename(columns={0: 'predicted'}, inplace=True)

df_table = confusion_matrix(test_calc['y'],test_calc['predicted'])
print (df_table)

print('accuracy:', (df_table[0,0] + df_table[1,1]) / (df_table[0,0] + df_table[0,1] + df_table[1,0] + df_table[1,1]))
print ('precision:', df_table[1,1] / (df_table[1,1] + df_table[0,1]))
print('recall:', df_table[1,1] / (df_table[1,1] + df_table[1,0]))

p = df_table[1,1] / (df_table[1,1] + df_table[0,1])
r = df_table[1,1] / (df_table[1,1] + df_table[1,0])
print('f1 score: ', (2*p*r)/(p+r))

[[ 272  186   16   26   65  180  110  202]
 [ 124  313   21   24  110  204  104  232]
 [  18   12   64   46    6   13    4    8]
 [   8    4   22  167    0   11    9   24]
 [  41  119    2    1  433  198   44   90]
 [  84  102    3    0   83  965  265  387]
 [  29   19    0    0   10  198  563  497]
 [  17   18    0    1    9  100  160 2957]]
accuracy: 0.653631284916
precision: 0.627254509018
recall: 0.716247139588
f1 score:  0.668803418803


# Output

In [14]:
y_df=pd.DataFrame(xgb.predict(x_df))
y_df.rename(columns={0: '44_df'}, inplace=True)
y_df.to_csv('44_df.csv')

In [15]:
x_sub = x_sub.drop('MH_24__14', 1).copy()
x_sub['II_3__8']=0
x_sub = x_sub[['BMI__1','BMI__2','BMI__3','BMI__4','BMI__5','BMI__6','BMI__7','BMI__8','BMI__9','BMI__10','BMI__11','BMI__12',
               'BMI__13','BMI__14','EI_1__1','EI_1__2','EI_1__3','EI_1__4','EI_1__5','EI_1__6','EI_1__7','EI_1__8','EI_1__9',
               'EI_1__10','EI_1__11','EI_1__12','EI_1__13','EI_1__14','EI_2__2','EI_2__7','EI_2__11','EI_2__12','EI_2__14',
               'EI_3__13','EI_4__12','EI_4__13','EI_4__14','EI_4__15','EI_5__13','EI_6__1','EI_6__2','EI_6__3','EI_6__4','EI_6__5',
               'EI_6__6','EI_6__7','EI_6__8','EI_6__9','EI_6__10','EI_6__11','EI_6__12','EI_6__13','EI_6__15','FH_1__2','FH_1__9',
               'FH_2__1','FH_2__2','FH_2__3','FH_2__4','FH_2__5','FH_2__6','FH_2__7','FH_2__8','FH_2__9','FH_2__10','FH_2__11','FH_2__12',
               'FH_2__13','FH_2__14','FH_2__15','FH_3__1','FH_3__2','FH_3__3','FH_3__4','FH_3__5','FH_3__6','FH_3__7','FH_3__8',
               'FH_3__9','FH_3__10','FH_3__11','FH_3__12','FH_3__13','FH_3__14','FH_3__15','FH_4__1','FH_4__2','FH_4__3','FH_4__4',
               'FH_4__5','FH_4__6','FH_4__7','FH_4__8','FH_4__9','FH_4__10','FH_4__11','FH_4__12','FH_4__13','FH_4__14','FH_4__15',
               'FH_5__1','FH_5__2','FH_5__3','FH_5__4','FH_5__5','FH_5__6','FH_5__7','FH_5__8','FH_5__9','FH_5__10','FH_5__11','FH_5__12',
               'FH_5__13','FH_5__14','FH_5__15','HT__1','HT__2','HT__4','HT__5','HT__6','HT__7','HT__8','HT__9','HT__11','HT__12','HT__13',
               'HT__14','IAGE__1','IAGE__2','IAGE__3','IAGE__4','IAGE__5','IAGE__6','IAGE__7','IAGE__8','IAGE__9','IAGE__10','IAGE__11','IAGE__12',
               'IAGE__13','IAGE__14','IH_1__9','IH_2__14','IH_3__10','IH_4__7','IH_4__11','IH_5__1','IH_5__2','IH_5__3','IH_5__4','IH_5__5',
               'IH_5__7','IH_5__8','IH_5__9','IH_5__10','IH_5__11','IH_5__12','IH_5__13','IH_5__14','IH_5__15','IH_7__8','IH_7__11','IH_8__7',
               'IH_8__12','IH_9__4','IH_9__11','II_1__13','II_1__14','II_2__14','II_3__3','II_3__5','II_3__7','II_3__8','II_3__11','II_3__13',
               'II_3__14','II_4__8','II_5__14','II_6__11','II_7__14','MH_1__2','MH_1__4','MH_1__6','MH_1__7','MH_1__8','MH_1__9','MH_1__10','MH_1__11',
               'MH_1__12','MH_1__13','MH_1__14','MH_1__15','MH_2__1','MH_2__3','MH_2__4','MH_2__5','MH_2__7','MH_2__8','MH_2__9','MH_2__10','MH_2__11',
               'MH_2__12','MH_2__13','MH_2__14','MH_3__14','MH_4__10','MH_5__14','MH_6__7','MH_7__7','MH_7__14','MH_8__7','MH_8__14','MH_9__9',
               'MH_10__3','MH_10__8','MH_10__15','MH_11__7','MH_12__14','MH_13__8','MH_14__7','MH_15__2','MH_15__3','MH_15__4',
               'MH_15__5','MH_15__6','MH_15__7','MH_15__8','MH_15__9','MH_15__10','MH_15__12','MH_15__15','MH_16__13','MH_17__7','MH_18__14',
               'MH_19__14','MH_20__7','MH_21__14','MH_22__7','MH_23__9','MH_24__4','MH_24__7','MH_24__9','MH_24__10','MH_24__11','MH_24__12',
               'MH_24__13','MH_24__15','MH_25__13','MH_25__14','MH_26__8','MH_27__7','MH_28__14','MH_29__9','MH_30__14','MH_31__7',
               'MH_32__11','MH_32__12','MH_32__14','MH_32__15','MH_33__8','MH_34__8','MH_35__14','MH_36__6','MH_36__13','MH_37__7',
               'MH_38__14','MH_39__8','MH_40__7','MH_41__12','MK_1__14','MK_2__14','MK_3__14','MK_4__14','MK_5__14','MK_6__14','MK_7__14',
               'MK_8__14','MK_9__14','MK_10__14','MK_11__14','MK_12__14','MK_13__14','MK_14__14','MK_15__13','MK_16__14','MK_17__14','MK_18__14',
               'MK_19__14','MK_20__14','MK_21__14','MK_22__14','MK_23__14','MK_24__14','MK_25__14','MK_26__14','MK_27__14','MK_28__14',
               'MK_29__14','MK_30__14','MK_31__14','MK_32__14','MK_33__14','MK_34__14','MK_35__14','MK_36__14','MK_37__14','MK_38__14',
               'MK_39__14','MK_40__14','MK_41__14','MK_42__14','MK_43__14','MK_44__14','MK_45__14','MK_46__14','MK_47__14','MK_48__14',
               'PI_1__14','PI_2__12','PI_2__13','PI_2__14','PI_2__15','PI_2__16','PI_2__17','PI_2__18','PI_2__21','PI_2__22','PI_2__31',
               'PI_2__32','PI_2__33','PI_2__34','PI_2__41','PI_2__42','PI_2__43','PI_2__44','PI_2__50','PI_3__1','PI_3__8','PI_3__14',
               'PI_4__2','PI_4__4','PI_4__5','PI_4__7','PI_4__8','PI_4__9','PI_4__10','PI_4__11','PI_4__12','PI_4__13','PI_4__14','PI_5__14',
               'PI_6__8','PI_7__14','WT__1','WT__2','WT__3','WT__4','WT__5','WT__6','WT__7','WT__8','WT__9','WT__10','WT__11','WT__12',
               'WT__13','WT__14']].copy()
Response=pd.DataFrame(xgb.predict(x_sub))
sub = pd.concat([y_sub,Response],axis=1)
sub.rename(columns={0: 'Response'}, inplace=True)
sub.to_csv('44_sub.csv',index = False)